# Addestramento 1

In [3]:
import tensorflow as tf
from keras.legacy_tf_layers.convolutional import Conv1D
from tensorflow import keras
import pandas as pd
!pip install d2l==1.0.0a1.post0
from d2l import tensorflow as d2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split

In [2]:
model = keras.models.Sequential([
keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="valid", input_shape=[300, 1]),
keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="valid"),
keras.layers.MaxPooling1D(pool_size=2, strides=2),
keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, padding="valid"),
keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, padding="valid"),
keras.layers.MaxPooling1D(pool_size=2, strides=2),
keras.layers.Flatten(),
keras.layers.Dense(256, activation=tf.nn.relu),
keras.layers.Dense(128, activation=tf.nn.relu),
keras.layers.Dense(5, activation=tf.nn.softmax)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 296, 64)           384       
                                                                 
 conv1d_1 (Conv1D)           (None, 292, 64)           20544     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 146, 64)          0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 144, 128)          24704     
                                                                 
 conv1d_3 (Conv1D)           (None, 142, 128)          49280     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 71, 128)          0         
 1D)                                                    

In [3]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x=x_train, y=y_train, epochs=3)

test_error_rate = model.evaluate(x_test, y_test, verbose=0)

NameError: name 'x_train' is not defined

# Addestramento 2

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model



input_tot = Input(shape=(251, 1), name ="Input_tot")

# Pipeline 1
branch1_1 = keras.layers.Conv1D(filters=8, kernel_size=4, activation='relu', name ="branch1_1")(input_tot)
branch1_2 = keras.layers.MaxPooling1D(pool_size=2, strides=2, name ="branch1_2")(branch1_1)
branch1_3 = keras.layers.Conv1D(filters=24, kernel_size=6, activation='relu', name ="branch1_3")(branch1_2)
branch1_4 = keras.layers.MaxPooling1D(pool_size=2, strides=2, name ="branch1_4")(branch1_3)

# Pipeline 2
branch2_1 = keras.layers.Conv1D(filters=8, kernel_size=6, activation='relu', name ="branch2_1")(input_tot)
branch2_2 = keras.layers.MaxPooling1D(pool_size=2, strides=2, name ="branch2_2")(branch2_1)
branch2_3 = keras.layers.Conv1D(filters=24, kernel_size=8, activation='relu', name ="branch2_3")(branch2_2)
branch2_4 = keras.layers.MaxPooling1D(pool_size=2, strides=2, name ="branch2_4")(branch2_3)

# Pipeline 3
branch3_1 = keras.layers.Conv1D(filters=8, kernel_size=8, activation='relu', name ="branch3_1")(input_tot)
branch3_2 = keras.layers.MaxPooling1D(pool_size=2, strides=2, name ="branch3_2")(branch3_1)
branch3_3 = keras.layers.Conv1D(filters=24, kernel_size=10, activation='relu', name ="branch3_3")(branch3_2)
branch3_4 = keras.layers.MaxPooling1D(pool_size=2, strides=2, name ="branch3_4")(branch3_3)

#Merging tre pipeline
branch_concatenate = concatenate([branch1_4,branch2_4,branch3_4], axis=1, name="concatenated_layer")

#Final Layer
dense1 = Dense(256, activation = "sigmoid", name = "dense1")(branch_concatenate)
dense2 = Dense(32, activation = "sigmoid", name = "dense2")(dense1)
output_layer = Dense(4, activation = "sigmoid", name = "output_layer")(dense2)

#Model Definition
merged = Model(inputs=[input_tot],outputs=[output_layer], name = "merged model")

#Model Details
merged.summary()
keras.utils.plot_model(merged, "output/architecture.png", show_shapes=True)

Model: "merged model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input_tot (InputLayer)         [(None, 251, 1)]     0           []                               
                                                                                                  
 branch1_1 (Conv1D)             (None, 248, 8)       40          ['Input_tot[0][0]']              
                                                                                                  
 branch2_1 (Conv1D)             (None, 246, 8)       56          ['Input_tot[0][0]']              
                                                                                                  
 branch3_1 (Conv1D)             (None, 244, 8)       72          ['Input_tot[0][0]']              
                                                                                       

In [8]:
df = pd.read_csv('heartbeats/heartbeats_datasets.csv')

df.head(1)

,signal,annotypes
0,[-0.265 -0.27 -0.265 -0.26 -0.275 -0.28 -0....,N


In [9]:
list(df['signal'][0])

['[',
 '-',
 '0',
 '.',
 '2',
 '6',
 '5',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '7',
 ' ',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '6',
 '5',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '6',
 ' ',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '7',
 '5',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '8',
 ' ',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '9',
 ' ',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '7',
 '5',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '7',
 ' ',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '6',
 '\n',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '8',
 ' ',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '8',
 ' ',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '8',
 '5',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '7',
 '5',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '7',
 '5',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '6',
 '5',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '7',
 ' ',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '8',
 '5',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '9',
 ' ',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '8',
 '\n',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '7',
 '5',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '8',
 '5',
 ' ',
 '-',
 '0',
 '.',
 '2',
 '8',
 ' ',
 ' ',
 '-',
 '0',
 '.',
 '3',
 '